# Chapter8 텍스트 분석 ~04 - 실리콘밸리 5차 정기세션 과제

## 스팸 텍스트 메시지 데이터 활용 링크: https://www.kaggle.com/team-ai/spam-text-message-classification/download

### 1. 텍스트 분석 수행 프로세스 및 텍스트 정규화 작업을 정리하세요

<텍스트 분석 수행 프로세스>
1. 텍스트 사전 준비작업(텍스트 전처리)
2. 피처 벡터화 / 추출
3. ML 모델 수립 및 학습/예측/평가

<텍스트 정규화 작업>
* 클렌징
* 토큰화
* 필터링/스톱 워드 제거/철자 수정
* Stemming
* Lemmatization

In [1]:
from nltk import sent_tokenize
from nltk import word_tokenize
import pandas as pd
import numpy as np

### 2. 해당 데이터의 index가 901인 문장에 대해 문장별 단어 토큰화(교재 함수 사용)를 수행하고 스톱 워드를 제거하세요

In [19]:
text_spam=pd.read_csv('spam.csv')
text_sample=text_spam.loc[901, 'Message']
text_sample

'Probably money worries. Things are coming due and i have several outstanding invoices for work i did two and three months ago.'

In [6]:
def tokenize_text(text):
    sentences = sent_tokenize(text)
    word_tokens = [word_tokenize(sentence) for sentence in sentences]
    return word_tokens
    
word_tokens = tokenize_text(text_sample)
word_tokens

[['Probably', 'money', 'worries', '.'],
 ['Things',
  'are',
  'coming',
  'due',
  'and',
  'i',
  'have',
  'several',
  'outstanding',
  'invoices',
  'for',
  'work',
  'i',
  'did',
  'two',
  'and',
  'three',
  'months',
  'ago',
  '.']]

In [10]:
import nltk
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english')
all_tokens = []
for sentence in word_tokens:
    filtered_words = []
    for word in sentence:
        word = word.lower()
        if word not in stopwords:
            filtered_words.append(word)
    all_tokens.append(filtered_words)

all_tokens

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\round\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


[['probably', 'money', 'worries', '.'],
 ['things',
  'coming',
  'due',
  'several',
  'outstanding',
  'invoices',
  'work',
  'two',
  'three',
  'months',
  'ago',
  '.']]

### 3.  해당 문장의 결과 토큰화 값들 중 진행형에 대해 Stemmimg과 Lemmatization을 수행 후 결과를 비교하세요

In [14]:
from nltk.stem import LancasterStemmer

stemmer = LancasterStemmer()
tokens = ['coming', 'outstanding']
tokens_stemmered = [stemmer.stem(x) for x in tokens]
tokens_stemmered

['com', 'outstand']

In [15]:
from nltk.stem import WordNetLemmatizer

lemma = WordNetLemmatizer()
print(lemma.lemmatize(tokens[0], 'v'))
print(lemma.lemmatize(tokens[1], 'a'))

come
outstanding


### 4. 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가
### 4-1. 위의 스팸 데이터로 train, test set을 나누세요 (힌트: train_test_split)

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(text_spam['Message'], text_spam['Category'], test_size=0.3, random_state=100)

### 4-2. 두 개의 피처 벡터화를 수행 후 각각의 벡터화 shape을 구하세요

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

cnt_vect = CountVectorizer()
cnt_vect.fit(X_train, y_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 Text의 CountVectorizer Shape:',X_train_cnt_vect.shape)

학습 데이터 Text의 CountVectorizer Shape: (3900, 7174)


In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

print('학습 데이터 Text의 TF-IDF Shape:',X_train_tfidf_vect.shape)

학습 데이터 Text의 TF-IDF Shape: (3900, 7174)


### 4-3. 각각에 로지스틱 회귀 모형을 적용해 예측해 보세요

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)

print('CountVectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

CountVectorized Logistic Regression 의 예측 정확도는 0.984


In [35]:
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)

print('TF-IDF Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Logistic Regression 의 예측 정확도는 0.970


C:\Users\round\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## Women's E-Commerce Clothing Reviews 데이터 링크: https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews

### 5. 파이프라인 사용 
### 5-1. 새로운 데이터 셋을 다운로드 후 'Review text'로 'Rating'을 예측하도록 적절한 전처리를 수행 후 test, train set으로 분할하세요(힌트: 결측치 처리-어떻게?)

In [48]:
e_review=pd.read_csv('woman_clothing.csv')
e_review.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [51]:
e_review=e_review.dropna(subset=['Review Text'])
X_train, X_test, y_train, y_test = train_test_split(e_review['Review Text'], e_review['Rating'], test_size=0.3, random_state=42)

### 5-2. 사이킷런의 Pipeline방식을 적용해 TF-IDF기반으로 벡터화하고 로지스틱 회귀 모형으로 예측해보세요(파라미터 튜닝은 디폴트로 해도 되고 맘대로 해보세요)

In [52]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_df=300)),
    ('lr_clf', LogisticRegression(C=10))
])

pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print('예측 정확도 : {}'.format(accuracy_score(y_test, pred)))

C:\Users\round\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\round\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


예측 정확도 : 0.6185779478875313
